<a href="https://colab.research.google.com/github/AnittaNJ/Sales_DemandForecasting_Project/blob/main/Sales_DemandForecasting_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Business Context**

The Nielsen BookScan service is the world’s largest continuous book sales tracking service in the world, operating in the UK, Ireland, Australia, New Zealand, India, South Africa, Italy, Spain, Mexico, Brazil, Poland, and Colombia. Nielsen BookScan collects transactional data at the point of sale, directly from tills and dispatch systems of all major book retailers. This ensures detailed and highly accurate sales information on which books are selling and at what price, giving clients the most up-to-date and relevant data. The Nielsen BookScan Total Consumer Market (TCM) data covers approximately 90% of all retail print book purchases in the UK. The remaining sites are specialised, such as gift shops, specialist booksellers, and tourist information centres.


Nielsen BookScan can be used to:
Monitor titles and authors against the competition and overall market.
Analyse pricing and discounting by format or category.
Gauge the success of marketing campaigns and promotions.
See which categories are growing and declining.
Learn what works in your market and how that might differ from other countries.

Nielsen BookScan sales data can be analysed by various criteria, including category, publisher, and format,
allowing users to see which genres are selling in which format. Users can track market trends to see which titles are driving the results, and patterns can easily be interpreted. In addition, the actual selling price is included. This inclusion makes it easier to identify trends for the level of discounting (e.g. by title, author, genre, format, region, and publisher) when analysing book sales.


**Project context**

Nielsen is seeking to invest in developing a new service aimed at small to medium-sized independent publishers. This service is aimed at supporting publishers in using historical sales data to make data-driven decisions about their future investment in new publications. Their publisher customers are interested in being able to make more accurate predictions of the overall sales profile post-publication for better stock control and initial investment, but they are also interested in understanding the useful economic life span that a title may have.

Nielsen is targeting small to medium-sized independent publishers as their research has shown that there is a strong demand for this insight, but businesses cannot invest in this infrastructure and would pay a premium to have access to quality-assured data and analysis in this area. Producing a new publication requires a significant upfront investment, and they would like to be able to more accurately identify books with strong long-term potential. More specifically, they are looking for titles with sales patterns that exhibit well-established seasonal patterns and positive trends that show potential great returns and to learn more about these types of publications. Nielsen will then apply this understanding to their commission and print volume strategy to be more successful in acquiring titles that have longevity. Additionally, this will enable them to deliver better returns by ensuring the correct stock levels in relation to demand and avoiding over or understocking, which can be costly.

You will notice that some titles experience fluctuations in sales due to various factors, such as increased media attention or cultural relevance (e.g. the recent resurgence of interest in George Orwell’s 1984). However, certain books endure over time and are often studied in academic settings for their deeper significance.

For this project, Nielsen has provided you with two data sets. The objective is to identify sales patterns that demonstrate seasonal trends or any other traits, providing insights to inform reordering, restocking, and reprinting decisions for various books (by their International Standard Book Number, or ISBN).

<br>

## **Objective**

Conduct a comprehensive analysis on selected books from Nielsen's data, identifying key sales patterns that exhibit clear seasonal trends or other distinctive characteristics. These insights will serve as a data-driven foundation for optimising procurement, re-ordering, and stocking decisions, ensuring efficient inventory management.

## Import Libraries

In [ ]:
!pip install pmdarima
!pip install -U keras-tuner
!pip install scikit-learn
!pip install tensorflow

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gdown
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import RMSprop, Adam
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

## Load Dataset

In [ ]:
destination = 'ISBN_List.xlsx'

# Construct the download URL
download_url = f''

# Download the file using gdown
gdown.download(download_url, destination, quiet=False)

In [ ]:
destination = 'Weekly_trend.xlsx'

# Construct the download URL
download_url = f''

# Download the file using gdown
gdown.download(download_url, destination, quiet=False)

### ISBN_List File

In [ ]:
# Importing the data file ISBN_List.xlsx into a dataframe.
dfs = pd.read_excel('ISBN_List.xlsx', sheet_name=None)

# Concatenate all sheets into a single dataframe
isbn_df = pd.concat(dfs.values(), ignore_index=True)

# Display the combined dataframe
isbn_df

In [ ]:
isbn_df.shape

In [ ]:
isbn_df.info()

### Weekly_Trend File

In [ ]:
# Importing the data file Weekly_trend.xlsx into a dataframe.
dfs = pd.read_excel('Weekly_trend.xlsx', sheet_name=None)

# Concatenate all sheets into a single dataframe
weekly_df = pd.concat(dfs.values(), ignore_index=True)

# Display the combined dataframe
weekly_df

In [ ]:
weekly_df.shape

In [ ]:
weekly_df.info()

## Data Investigation and Preprocessing

In [ ]:
# Resample to weekly frequency ending on saturday, mean of the 'Volume' column
weekly_df = weekly_df.set_index('End Date')
weekly_sat_df = (
    weekly_df
    .groupby(['ISBN', 'Title'])  # Group by ISBN and Title
    .resample('W-SAT')           # Resample to weekly frequency ending on Saturday
    .agg({
        'Volume': 'sum'           # Sum the 'Volume' column to get total sales per week
    })
    .fillna(0)
    .reset_index()                # Reset index to keep 'ISBN' and 'Title' as columns
)

# Display the final DataFrame
weekly_sat_df


In [ ]:
# info about dataframe
weekly_sat_df.info()

In [ ]:
# Convert the ISBNs to a string type
weekly_sat_df['ISBN'] = weekly_sat_df['ISBN'].astype(str)

In [ ]:
# Convert the 'End Date' column to datetime object type
weekly_sat_df['End Date'] = pd.to_datetime(weekly_sat_df['End Date'])

In [ ]:
# Filtering out the ISBNs wherein sales data exists beyond 2024-07-01.
isbns_beyond = weekly_sat_df[weekly_sat_df['End Date'] > '2024-07-01']['ISBN'].unique()
for i, isbn in enumerate(isbns_beyond):
    print(f"{i+1}. {isbn}")

In [ ]:
# Plot the data of each ISBNs from the previous step using loop

import matplotlib.pyplot as plt

for isbn in isbns_beyond:
  df_temp = weekly_sat_df[weekly_sat_df['ISBN'] == isbn]
  book_title = df_temp['Title'].iloc[0]
  plt.figure(figsize=(12, 6))
  plt.plot(df_temp['End Date'], df_temp['Volume'])
  plt.title(f"Sales data for ISBN {isbn} - {book_title}")
  plt.xlabel('Date')
  plt.ylabel('Volume')
  plt.grid(True)
  plt.show()


The Alchemist and The Very Hungry Caterpillar were selected to perform further analysis.

In [ ]:
# creating a dataframe to include The Alchemist and The Very Hungry Caterpillar books.
books_of_interest = ['Alchemist, The', 'Very Hungry Caterpillar, The']
df_books_of_interest= weekly_sat_df[weekly_sat_df['Title'].isin(books_of_interest)]
df_books_of_interest

In [ ]:
# Filter the sales data for both these books to retain the date range >2012-01-01, until the final datapoint.
df_books_of_interest = df_books_of_interest[df_books_of_interest['End Date'] > '2012-01-01']
df_books_of_interest

In [ ]:
# information about df_books_of_interest dataframe
df_books_of_interest.info()

In [ ]:
# setting the 'End Date' as the index
df_books_of_interest = df_books_of_interest.set_index(['End Date'])
df_books_of_interest

In [ ]:
# created a separate dataframe for each book.
alchemist = df_books_of_interest[df_books_of_interest['Title'] == 'Alchemist, The']
caterpillar = df_books_of_interest[df_books_of_interest['Title'] == 'Very Hungry Caterpillar, The']

In [ ]:
# checking the sales pattern of each book.
import pandas as pd
import matplotlib.pyplot as plt

# Plotting the time series for The Alchemist
plt.figure(figsize=(14, 6))
plt.subplot(2, 1, 1)
plt.plot(alchemist.index, alchemist['Volume'], label='Sales', color='blue')
plt.title("Sales Data for 'The Alchemist'")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.grid()

# Plotting the time series for The Very Hungry Caterpillar
plt.subplot(2, 1, 2)
plt.plot(caterpillar.index, caterpillar['Volume'], label='Sales', color='green')
plt.title("Sales Data for 'The Very Hungry Caterpillar'")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


## Decomposition

In [ ]:
for book_title in books_of_interest:
  df_book = df_books_of_interest[df_books_of_interest['Title'] == book_title]
  sales = df_book['Volume']
  # Check if the DataFrame is not empty and has enough data points
  if not df_book.empty and len(sales) >= 24:
      print(f"\nDecomposition for: {book_title}")

      # Determine whether to use additive or multiplicative decomposition
      # First, attempt additive decomposition
      try:
          decomposition = seasonal_decompose(sales, model='additive', period=52)

          # Plot the decomposition components
          plt.figure(figsize=(12, 8))
          plt.subplot(411)
          plt.plot(df_book.index, sales, label='Original', color='blue')
          plt.title(f'Original Sales Data for {book_title}')
          plt.legend(loc='best')

          plt.subplot(412)
          plt.plot(decomposition.trend, label='Trend', color='orange')
          plt.title('Trend Component')
          plt.legend(loc='best')

          plt.subplot(413)
          plt.plot(decomposition.seasonal, label='Seasonality', color='green')
          plt.title('Seasonal Component')
          plt.legend(loc='best')

          plt.subplot(414)
          plt.plot(decomposition.resid, label='Residuals', color='red')
          plt.title('Residual Component')
          plt.legend(loc='best')

          plt.tight_layout()
          plt.show()

      except ValueError as e:
          print(f"Error during additive decomposition for {book_title}: {e}")

      # Check for zero or negative values in sales
      if (sales <= 0).any():
          print(f"Additive decomposition is enforced due to zero or negative values in {book_title}.")
      else:
          # If residual standard deviation indicates multiplicative is better, try that
          if decomposition.resid.std() > 0.5 * sales.std():
              print(f"Using multiplicative decomposition for {book_title}.")

              # Perform multiplicative decomposition
              decomposition = seasonal_decompose(sales, model='multiplicative', period=52)

              # Plot the decomposition components for multiplicative
              plt.figure(figsize=(12, 8))
              plt.subplot(411)
              plt.plot(df_book.index, sales, label='Original', color='blue')
              plt.title(f'Original Sales Data for {book_title}')
              plt.legend(loc='best')

              plt.subplot(412)
              plt.plot(decomposition.trend, label='Trend', color='orange')
              plt.title('Trend Component (Multiplicative)')
              plt.legend(loc='best')

              plt.subplot(413)
              plt.plot(decomposition.seasonal, label='Seasonality', color='green')
              plt.title('Seasonal Component (Multiplicative)')
              plt.legend(loc='best')

              plt.subplot(414)
              plt.plot(decomposition.resid, label='Residuals', color='red')
              plt.title('Residual Component (Multiplicative)')
              plt.legend(loc='best')

              plt.tight_layout()
              plt.show()

          else:
              print(f"Additive decomposition appears suitable for {book_title}.")
  else:
      print(f"Not enough data for decomposition for {book_title}.")


## ACF & PACF plots

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


# ACF Plot for The Alchemist
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plot_acf(alchemist['Volume'], lags=20, ax=plt.gca())
plt.title('ACF for The Alchemist')

# PACF Plot for The Alchemist
plt.subplot(2, 1, 2)
plot_pacf(alchemist['Volume'], lags=20, ax=plt.gca())
plt.title('PACF for The Alchemist')

plt.tight_layout()
plt.show()


# ACF Plot for The Very Hungry Caterpillar
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plot_acf(caterpillar['Volume'], lags=20, ax=plt.gca())
plt.title('ACF for The Very Hungry Caterpillar')

# PACF Plot for The Very Hungry Caterpillar
plt.subplot(2, 1, 2)
plot_pacf(caterpillar['Volume'], lags=20, ax=plt.gca())
plt.title('PACF for The Very Hungry Caterpillar')

plt.tight_layout()
plt.show()


## Stationarity

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Function to perform ADF test and print results
def check_stationarity(series, title):
    adf_test = adfuller(series)
    print(f"Results of ADF Test for {title}:")
    print(f"ADF Statistic: {adf_test[0]}")
    print(f"p-value: {adf_test[1]}")
    print("Critical Values:")
    for key, value in adf_test[4].items():
        print(f"   {key}: {value}")

    # Interpretation
    if adf_test[1] <= 0.05:
        print(f"{title} is stationary (reject null hypothesis)")
    else:
        print(f"{title} is not stationary (fail to reject null hypothesis)")
    print("\n")

# For The Alchemist
check_stationarity(alchemist['Volume'], "The Alchemist")

# For The Very Hungry Caterpillar
check_stationarity(caterpillar['Volume'], "The Very Hungry Caterpillar")


In [ ]:
# From the previous dataframe 'caterpillar' of book 'very hungry caterpillar'
# Perform differencing on the 'Volume' column and overwrite it
caterpillar['Volume'] = caterpillar['Volume'].diff()

# Drop missing values that result from differencing
caterpillar = caterpillar.dropna()

# Display the resulting dataframe
print(caterpillar)

Now check stationarity after differencing of Very Hungry Caterpillar series.

In [ ]:
# Check for stationarity with the ADF test
adf_result = adfuller(caterpillar['Volume'])  # Perform ADF on the non-null values
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')

# If p-value < 0.05, the data is stationary
if adf_result[1] > 0.05:
    print("Series is still not stationary. Consider further transformations.")
else:
    print("Series is stationary.")

## ARIMA

### The Alchemist

Performing Auto ARIMA on 'The Alchemist' book.

In [ ]:
# Forecast horizon
forecast_horizon = 32

def auto_arima_forecast(df_book, book_title):
    print(f"\nAuto ARIMA for {book_title}")
    sales = df_book['Volume']

    # Split the data into training and testing sets
    train_data = sales[:-forecast_horizon]
    test_data = sales[-forecast_horizon:]

    # Perform Auto ARIMA model selection
    model_auto_arima = auto_arima(
        train_data,
        seasonal=True,
        m=52,                               # Weekly seasonality m = 52
        max_p=5, max_q=5, max_P=5, max_Q=5, # Upper bounds for p, q, P, Q set to 5
        d=0,                                # Differencing order
        max_d=2,
        start_p=1, start_q=1, start_P=0, start_Q=0,
        stepwise=True,
        trace=True,
        error_action='ignore',
        suppress_warnings=True,
    )

    # Get the best model parameters
    print(f"Best ARIMA Model: {model_auto_arima.order}")

    # Fit the best ARIMA model on the training data
    model = ARIMA(train_data, order=model_auto_arima.order)
    model_fit = model.fit()

    # Print model summary
    print(model_fit.summary())

    # Get fitted values for the training period
    fitted_values = model_fit.fittedvalues  # These are the in-sample predictions

    # Calculate residuals for the training period (actual - fitted)
    residuals_train = train_data - fitted_values

    # Plot residuals (training period)
    plt.figure(figsize=(12, 6))
    plt.plot(train_data.index, residuals_train, label='Training Residuals', color='orange')
    plt.title(f"Residuals for {book_title}")
    plt.xlabel('Date')
    plt.ylabel('Residual')
    plt.axhline(y=0, color='black', linestyle='--')
    plt.legend()
    plt.show()

    # Forecast the final 32 weeks (true forecasting, future values)
    forecast_results = model_fit.get_forecast(steps=forecast_horizon)

    # Get forecasted values and confidence intervals for future periods
    forecast = forecast_results.predicted_mean
    conf_int = forecast_results.conf_int()

    # Create a date range for the forecast (future dates)
    forecast_index = pd.date_range(start=sales.index[-1] + pd.Timedelta(weeks=1), periods=forecast_horizon, freq='W-SAT')

    # Calculate MAE and MAPE
    mae = mean_absolute_error(test_data, forecast)
    mape = mean_absolute_percentage_error(test_data, forecast) * 100

    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    # Print actual vs forecasted values
    actual_vs_forecast = pd.DataFrame({
        'Actual': test_data.values,
        'Forecasted': forecast.values,
        'Forecast date': forecast_index
    })
    print("\nActual vs Forecasted Values:")
    print(actual_vs_forecast)

    # Plot the actual sales, forecast
    plt.figure(figsize=(12, 6))

    # Plot the original data (actual sales)
    plt.plot(sales, label='Actual Sales', color='blue')

    # Plot forecast (out-of-sample)
    plt.plot(forecast_index, forecast, label='Forecast', color='green')

    # Plot confidence intervals
    plt.fill_between(forecast_index,
                     conf_int.iloc[:, 0], conf_int.iloc[:, 1],
                     color='lightgreen', alpha=0.5, label='95% Confidence Interval')

    plt.title(f"Actual Sales and Forecast for {book_title}")
    plt.xlabel('Date')
    plt.ylabel('Sales Volume')
    plt.legend()
    plt.grid()
    plt.show()

    return model_fit

# Perform Auto ARIMA and forecasting for "The Alchemist"
model_alchemist = auto_arima_forecast(alchemist,'The Alchemist')

### The Very Hungry Caterpillar

Note : The separate function was created for this book to set parameter range (p,q = 2), as the system RAM was crashing when parameters upper bound were (p ,q = 5 )increased for this book.

In [ ]:
# Forecast horizon
forecast_horizon = 32

def auto_arima_forecast(df_book, book_title):
    print(f"\nAuto ARIMA for {book_title}")
    sales = df_book['Volume']

    # Split the data into training and testing sets
    train_data = sales[:-forecast_horizon]
    test_data = sales[-forecast_horizon:]

    # Perform Auto ARIMA model selection
    model_auto_arima = auto_arima(
        train_data,
        seasonal=True,
        m=52,                               # Weekly seasonality m = 52
        max_p=2, max_q=2, max_P=2, max_Q=2, # Upper bounds for p, q, P, Q is set to 2
        d=0,                                # Differencing order
        max_d=2,
        start_p=1, start_q=1, start_P=0, start_Q=0,
        stepwise=True,
        trace=True,
        error_action='ignore',
        suppress_warnings=True,
    )

    # Get the best model parameters
    print(f"Best ARIMA Model: {model_auto_arima.order}")

    # Fit the best ARIMA model on the training data
    model = ARIMA(train_data, order=model_auto_arima.order)
    model_fit = model.fit()

    # Print model summary
    print(model_fit.summary())

    # Get fitted values for the training period
    fitted_values = model_fit.fittedvalues  # These are the in-sample predictions

    # Calculate residuals for the training period (actual - fitted)
    residuals_train = train_data - fitted_values

    # Plot residuals (training period)
    plt.figure(figsize=(12, 6))
    plt.plot(train_data.index, residuals_train, label='Training Residuals', color='orange')
    plt.title(f"Residuals for {book_title}")
    plt.xlabel('Date')
    plt.ylabel('Residual')
    plt.axhline(y=0, color='black', linestyle='--')
    plt.legend()
    plt.show()

    # Forecast the final 32 weeks (true forecasting, future values)
    forecast_results = model_fit.get_forecast(steps=forecast_horizon)

    # Get forecasted values and confidence intervals for future periods
    forecast = forecast_results.predicted_mean
    conf_int = forecast_results.conf_int()

    # Create a date range for the forecast (future dates)
    forecast_index = pd.date_range(start=sales.index[-1] + pd.Timedelta(weeks=1), periods=forecast_horizon, freq='W-SAT')

    # Calculate MAE and MAPE
    mae = mean_absolute_error(test_data, forecast)
    mape = mean_absolute_percentage_error(test_data, forecast) * 100

    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    # Print actual vs forecasted values
    actual_vs_forecast = pd.DataFrame({
        'Actual': test_data.values,
        'Forecasted': forecast.values,
        'Forecast date': forecast_index
    })
    print("\nActual vs Forecasted Values:")
    print(actual_vs_forecast)

    # Plot the actual sales, forecast
    plt.figure(figsize=(12, 6))

    # Plot the original data (actual sales)
    plt.plot(sales, label='Actual Sales', color='blue')

    # Plot forecast (out-of-sample)
    plt.plot(forecast_index, forecast, label='Forecast', color='green')

    # Plot confidence intervals
    plt.fill_between(forecast_index,
                     conf_int.iloc[:, 0], conf_int.iloc[:, 1],
                     color='lightgreen', alpha=0.5, label='95% Confidence Interval')

    plt.title(f"Actual Sales and Forecast for {book_title}")
    plt.xlabel('Date')
    plt.ylabel('Sales Volume')
    plt.legend()
    plt.grid()
    plt.show()

    return model_fit
# Perform Auto ARIMA and forecasting for "Very Hungry Caterpillar, The"
model_caterpillar = auto_arima_forecast(caterpillar,'Very Hungry Caterpillar, The')

# ML & DL models

## XGBoost

Next trying out ML model, by creating the required pipeline for the XGBoost model. Performing cross - validation and parameter tuning (including window_length) using grid search. The forecast horizon is 32 weeks.

A function was created for both books, by setting window length and parameter value range.

In [ ]:
# Create a function to create lagged features
def create_lagged_features(df, window_length):
    """Create lag features for the time series."""
    df_lagged = pd.DataFrame()
    for i in range(1, window_length + 1):
        df_lagged[f'lag_{i}'] = df['Volume'].shift(i)
    df_lagged['Volume'] = df['Volume']
    return df_lagged.dropna()

# Custom function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scaling the data
    ('xgb', XGBRegressor())  # XGBoost Regressor
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'xgb__n_estimators': [100, 200],  # Number of trees
    'xgb__learning_rate': [0.01, 0.02, 0.03, 0.1],  # Learning rate
    'xgb__max_depth': [3, 4, 5, 6],  # Maximum depth of a tree
    'xgb__subsample': [0.7, 0.8, 0.9, 1.0],  # Subsampling ratio
    'xgb__colsample_bytree': [0.7, 0.8, 0.9, 1.0],  # Subsampling ratio for columns
}

# Custom scoring function combining MAE and MAPE
scoring = {
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'MAPE': make_scorer(mean_absolute_percentage_error, greater_is_better=False)
}

# Function to perform grid search and forecast
def xgboost_forecast(df, book_title, window_lengths=[1, 2, 3, 4, 5]):
    print(f"\nTraining and forecasting for {book_title}")

    sales = df['Volume'].dropna()

    # Split data into training and testing
    train_data = sales[:-32]
    test_data = sales[-32:]

    best_score = float('inf')
    best_model = None
    best_window_length = None
    best_y_test = None
    best_y_pred = None

    for window_length in window_lengths:
        # Create lagged features for the training data
        df_train_lagged = create_lagged_features(train_data.to_frame(), window_length)
        X_train = df_train_lagged.drop('Volume', axis=1)
        y_train = df_train_lagged['Volume']

        # TimeSeriesSplit for cross-validation
        tscv = TimeSeriesSplit(n_splits=5)

        # Grid search with cross-validation
        grid_search = GridSearchCV(pipeline, param_grid, cv=tscv, scoring=scoring, refit='MAE', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Best model for the current window length
        best_model_current = grid_search.best_estimator_
        best_window_length_current = window_length

        # Create lagged features for the test data
        df_test_lagged = create_lagged_features(test_data.to_frame(), window_length)
        X_test = df_test_lagged.drop('Volume', axis=1)
        y_test = df_test_lagged['Volume']

        # Predict on the test data
        y_pred = best_model_current.predict(X_test)

        # Calculate MAE and MAPE
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)

        print(f"Window Length: {window_length} | MAE: {mae:.2f} | MAPE: {mape:.2f}%")


        # Store the best model and predictions if current one is better
        if mae < best_score:
            best_score = mae
            best_model = best_model_current
            best_window_length = best_window_length_current
            best_y_test = y_test
            best_y_pred = y_pred


    # Display expected and predicted values for the best window length
    print(f"\nExpected vs Predicted values for {book_title}:")
    expected_vs_predicted = pd.DataFrame({
        'Expected': best_y_test.values,
        'Predicted': best_y_pred
    })
    print(expected_vs_predicted)

    # Best MAE and MAPE
    best_mape = mean_absolute_percentage_error(best_y_test, best_y_pred)
    print(f"\nBest Window Length: {best_window_length} | Best MAE: {best_score:.2f} | Best MAPE: {best_mape:.2f}%")


    # Final forecasting on the test set using the best model
    print(f"\nBest window length for {book_title}: {best_window_length}")
    df_final_lagged = create_lagged_features(sales.to_frame(), best_window_length)
    X_final = df_final_lagged.drop('Volume', axis=1)
    y_final = df_final_lagged['Volume']

    forecast = best_model.predict(X_final[-32:])
     # Generate future date indexes for the forecast
    forecast_index = pd.date_range(start=sales.index[-1] + pd.Timedelta(weeks=1), periods=32, freq='W')

    # Create a DataFrame for the forecast with future dates
    forecast_df = pd.DataFrame({
        'Forecast Date': forecast_index,
        'Forecasted data': forecast
    })

    # Plot the actual sales, predictions, and forecast
    plt.figure(figsize=(12, 6))

    # Plot the original data (actual sales)
    plt.plot(sales.index, sales.values, label='Original Data', color='blue')

    # Plot the forecasted values
    forecast_index = X_final.index[-len(forecast):]
    plt.plot(forecast_index, forecast, label='Forecasted Values', color='green')

    plt.title(f"Actual Sales and Forecast for {book_title}")
    plt.xlabel('Time')
    plt.ylabel('Sales Volume')
    plt.legend()
    plt.show()

    return best_model,forecast_df

### The Alchemist

In [ ]:
# Example usage for "The Alchemist"
xgboost_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Alchemist, The'], 'The Alchemist')

### The Very Hungry Caterpillar

Running XGBoost_forecast function for this book.

In [ ]:
# "The Very Hungry Caterpillar"
xgboost_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Very Hungry Caterpillar, The'], 'The Very Hungry Caterpillar')

## LSTM

Next is Creating an LSTM model for both books.
Using KerasTuner, and performing hyperparameter tuning using the training data, for both books to predict sales data of final 32 weeks.

In [ ]:
# Define hyperparameters
n_input = 52  # Number of previous timesteps to use
n_features = 1  # Number of features
forecast_horizon = 32

# Function to handle outliers using IQR method
def handle_outliers(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Replace outliers with NaN
    data[data < lower_bound] = np.nan
    data[data > upper_bound] = np.nan
    # Fill NaN values with the median
    return data.fillna(data.median())

# Function to prepare data for LSTM
def prepare_data_lstm(sales_train, n_input, n_features):
    scaler = StandardScaler()

    # Scale the data
    scaled_data = scaler.fit_transform(sales_train.values.reshape(-1, 1))

    # Create a TimeseriesGenerator
    generator = TimeseriesGenerator(scaled_data, scaled_data, length=n_input, batch_size=32)

    return generator, scaler

# Define the LSTM model
def build_model(hp):
    model = Sequential()

    # Add LSTM layers
    for i in range(hp.Int('num_lstm_layers', 1, 3)):
        model.add(LSTM(units=hp.Int(f'units_lstm_{i}', min_value=32, max_value=128, step=32),
                       activation='relu',
                       return_sequences=(i < hp.Int('num_lstm_layers', 1, 3) - 1),
                       input_shape=(n_input, n_features) if i == 0 else None))
        model.add(Dropout(hp.Float(f'dropout_lstm_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    # Dense output layer
    model.add(Dense(1))

    # Compile the model
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop'])
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    if optimizer == 'adam':
        model.compile(optimizer='adam', loss='mse')
    else:
        model.compile(optimizer='rmsprop', loss='mse')

    return model

# Function to forecast using the best LSTM model
def forecast_lstm(model, df, scaler, n_input, n_features, forecast_horizon):
    forecast = []

    # Get the last n_input data points, scaled
    input_seq = scaler.transform(df.values[-n_input:].reshape(-1, n_features))

    for _ in range(forecast_horizon):
        input_seq = input_seq.reshape((1, n_input, n_features))  # Reshape for LSTM input
        prediction = model.predict(input_seq, verbose=0)  # Make prediction
        forecast.append(prediction[0, 0])  # Append prediction to forecast

        # Update the input sequence for the next prediction
        input_seq = np.append(input_seq[:, 1:, :], prediction.reshape((1, 1, n_features)), axis=1)

    return scaler.inverse_transform(np.array(forecast).reshape(-1, 1))  # Reverse scaling

def tune_lstm_model(generator):
    tuner = RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=10,  # Increased number of trials to explore more combinations
        executions_per_trial=1,
        directory='lstm_tuner',
        project_name='lstm_sales_extended')

    # Run the search
    tuner.search(generator, epochs=100, validation_data=generator, verbose=1)

    # Get the best model and trial info
    best_model = tuner.get_best_models(num_models=1)[0]  # Return the best model
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]  # Get best trial

    # Print trial number and parameters
    print(f"Best trial number: {best_trial.trial_id}")
    print("Best hyperparameters:")
    for param, value in best_trial.hyperparameters.values.items():
        print(f"{param}: {value}")

    return best_model


# Function to train and predict sales for both books
def lstm_forecast(df, book_title):
    print(f"\nTraining and forecasting for {book_title}")

    # Handle outliers in the sales data
    df['Volume'] = handle_outliers(df['Volume'])

    sales = df['Volume']
    sales_train = sales[:-forecast_horizon]  # Training data
    sales_test = sales[-forecast_horizon:]

    # Prepare the data for LSTM
    generator, scaler = prepare_data_lstm(sales_train, n_input, n_features)

    # Tune hyperparameters using KerasTuner
    best_model = tune_lstm_model(generator)

    # Train the model
    history = best_model.fit(generator, epochs=100, validation_data=generator,
                             callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

    # Forecast the next 32 weeks
    forecast = forecast_lstm(best_model, sales, scaler, n_input, n_features, forecast_horizon)

    # Calculate MAE and MAPE
    mae = mean_absolute_error(sales_test, forecast)
    mape = mean_absolute_percentage_error(sales_test, forecast) * 100  # Convert to percentage

    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    # Plot the actual sales and forecast
    plt.figure(figsize=(12, 6))

    # Plot original data
    plt.plot(sales.index, sales.values, label='Actual Sales', color='blue')

    # Plot forecast
    future_index = pd.date_range(start=sales.index[-1], periods=forecast_horizon + 1, freq='W-SAT')[1:]
    plt.plot(future_index, forecast, label='Forecasted Sales', color='green')

    plt.title(f"Sales Forecast for {book_title}")
    plt.xlabel("Date")
    plt.ylabel("Sales Volume")
    plt.legend()
    plt.show()

    # Show expected vs forecast values
    comparison_df = pd.DataFrame({'Actual Sales': sales_test.values, 'Forecasted Sales': forecast.flatten()})

    return comparison_df, best_model

### The Alchemist

In [ ]:
# The Alchemist
lstm_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Alchemist, The'], 'The Alchemist')

### The Very Hungry Caterpillar

In [ ]:
# The Very Hungry Caterpillar
lstm_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Very Hungry Caterpillar, The'], 'Very Hungry Caterpillar, The')

# Hybrid Models

## SARIMA and LSTM in sequential combination

Next creating a hybrid model of SARIMA and LSTM in sequential combination wherein the residuals from SARIMA will be forecasted using LSTM.
The final prediction will be the sum of the predictions from SARIMA and LSTM. The LSTM will be trained on the residuals obtained during the training of the SARIMA model.
The forecast horizon will be the final 32 weeks.
Using KerasTuner to get the best model.


In [ ]:
# Define hyperparameters
n_input = 52  # Number of previous timesteps to use for LSTM
n_features = 1
forecast_horizon = 32

# Function to prepare data for LSTM
def prepare_data_lstm(residuals, n_input, n_features):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(residuals.values.reshape(-1, 1))

    generator = TimeseriesGenerator(scaled_data, scaled_data, length=n_input, batch_size=32)

    return generator, scaler

# Define the LSTM model
def build_lstm_model(hp):
    model = Sequential()
    # Add LSTM layers
    for i in range(hp.Int('num_lstm_layers', 1, 3)):
        model.add(LSTM(units=hp.Int(f'units_lstm_{i}', min_value=32, max_value=128, step=32),
                       activation='relu',
                       return_sequences=(i < hp.Int('num_lstm_layers', 1, 3) - 1),
                       input_shape=(n_input, n_features) if i == 0 else None))
        model.add(Dropout(hp.Float(f'dropout_lstm_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    # Dense output layer
    model.add(Dense(1))

    # Compile the model
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop'])
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    if optimizer == 'adam':
        model.compile(optimizer='adam', loss='mse')
    else:
        model.compile(optimizer='rmsprop', loss='mse')

    return model

# Function to run hyperparameter tuning using KerasTuner
def tune_lstm_model(generator):
    tuner = RandomSearch(
        build_lstm_model,
        objective='val_loss',
        max_trials=10,
        executions_per_trial=1,
        directory='lstm_tuner',
        project_name='lstm_sales')

    tuner.search(generator, epochs=50, validation_data=generator)

    # Get the best model and trial info
    best_model = tuner.get_best_models(num_models=1)[0]  # Return the best model
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]  # Get best trial

    # Print best model and trial information
    print("Best model summary:")
    best_model.summary()

    print(f"\nBest trial number: {best_trial.trial_id}")
    print("Best hyperparameters:")
    for param, value in best_trial.hyperparameters.values.items():
        print(f"{param}: {value}")

    return best_model

# Function to handle outliers using the IQR method
def handle_outliers(df):
    Q1 = df['Volume'].quantile(0.25)
    Q3 = df['Volume'].quantile(0.75)
    IQR = Q3 - Q1

    # Identify outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace outliers with the median
    df['Volume'] = np.where(
        (df['Volume'] < lower_bound) | (df['Volume'] > upper_bound),
        df['Volume'].median(),
        df['Volume']
    )
    return df

# Function to forecast using the LSTM model
def forecast_lstm(model, residuals, scaler, n_input, n_features, forecast_horizon):
    forecast = []
    input_seq = scaler.transform(residuals.values[-n_input:].reshape(-1, 1))  # Get last n_input data points

    for _ in range(forecast_horizon):
        input_seq = input_seq.reshape((1, n_input, n_features))  # Reshape for LSTM input
        prediction = model.predict(input_seq, verbose=0)  # Make prediction
        forecast.append(prediction[0, 0])  # Append prediction to forecast

        # Update the input sequence for the next prediction
        input_seq = np.append(input_seq[:, 1:, :], prediction.reshape((1, 1, n_features)), axis=1)  # Append predicted value

    return scaler.inverse_transform(np.array(forecast).reshape(-1, 1))  # Reverse scaling

# Function to hybrid forecasting with SARIMA and LSTM
def hybrid_forecast(df, book_title):
    print(f"\nTraining and forecasting for {book_title}")

    # Handle outliers in the sales data
    df = handle_outliers(df)

    sales = df['Volume']
    sales_train = sales[:-forecast_horizon]  # Training data
    sales_test = sales[-forecast_horizon:]   # Test data

    # Fit SARIMA model
    sarima_model = sm.tsa.statespace.SARIMAX(sales_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 52))
    sarima_fit = sarima_model.fit(disp=False)

    # Get SARIMA predictions and residuals
    sarima_pred = sarima_fit.predict(start=len(sales_train), end=len(sales_train) + forecast_horizon - 1)
    residuals = sales_train - sarima_fit.fittedvalues

    # Prepare residuals for LSTM
    generator, scaler = prepare_data_lstm(residuals, n_input, n_features)

    # Tune hyperparameters for LSTM
    best_lstm_model = tune_lstm_model(generator)

    # Train LSTM model
    best_lstm_model.fit(generator, epochs=100, validation_data=generator)

    # Forecast the next 32 weeks with LSTM
    lstm_forecast_residuals = forecast_lstm(best_lstm_model, residuals, scaler, n_input, n_features, forecast_horizon)

    # Final forecast combining SARIMA and LSTM
    final_forecast = sarima_pred.values + lstm_forecast_residuals.flatten()

    # Calculate MAE and MAPE
    mae = mean_absolute_error(sales_test, final_forecast)
    mape = mean_absolute_percentage_error(sales_test, final_forecast) * 100  # Convert to percentage

    print(f"\nMean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    # Plot the actual sales and final forecast
    plt.figure(figsize=(12, 6))

    # Plot original data
    plt.plot(sales.index, sales.values, label='Actual Sales', color='blue')

    # Plot final forecast
    future_index = pd.date_range(start=sales.index[-1], periods=forecast_horizon + 1, freq='W-SAT')[1:]
    plt.plot(future_index, final_forecast, label='Hybrid Forecast', color='green')

    plt.title(f"Sales Forecast for {book_title} using Hybrid SARIMA-LSTM Model")
    plt.xlabel("Date")
    plt.ylabel("Sales Volume")
    plt.legend()
    plt.show()

    # Show expected vs predicted values
    comparison_df = pd.DataFrame({'Actual Sales': sales_test.values, 'Forecasted Sales': final_forecast})
    print(comparison_df)

### The Alchemist

In [ ]:
# "The Alchemist"
hybrid_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Alchemist, The'], 'The Alchemist')

### The Very Hungry Caterpillar

In [ ]:
# "The Very Hungry Caterpillar"
hybrid_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Very Hungry Caterpillar, The'], 'Very Hungry Caterpillar, The')

## SARIMA and LSTM in parallel combination

Next creating a hybrid model of SARIMA and LSTM in parallel combination wherein the residuals from SARIMA will be forecasted using LSTM.
The final prediction will be the sum of the predictions from SARIMA and LSTM. The LSTM will be trained on the residuals obtained during the training of the SARIMA model.
The forecast horizon will be the final 32 weeks.
Using KerasTuner to get the best model.

In [ ]:
# Define hyperparameters
n_input = 52  # Number of previous timesteps to use for LSTM
n_features = 1
forecast_horizon = 32

# Function to prepare data for LSTM
def prepare_data_lstm(sales, n_input, n_features):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(sales.values.reshape(-1, 1))

    generator = TimeseriesGenerator(scaled_data, scaled_data, length=n_input, batch_size=32)

    return generator, scaler

# Define the LSTM model
def build_lstm_model(hp):
    model = Sequential()
    # Add LSTM layers
    for i in range(hp.Int('num_lstm_layers', 1, 3)):
        model.add(LSTM(units=hp.Int(f'units_lstm_{i}', min_value=32, max_value=128, step=32),
                       activation='relu',
                       return_sequences=(i < hp.Int('num_lstm_layers', 1, 3) - 1),
                       input_shape=(n_input, n_features) if i == 0 else None))
        model.add(Dropout(hp.Float(f'dropout_lstm_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    # Dense output layer
    model.add(Dense(1))

    # Compile the model
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop'])
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    if optimizer == 'adam':
        model.compile(optimizer='adam', loss='mse')
    else:
        model.compile(optimizer='rmsprop', loss='mse')

    return model


# Function to run hyperparameter tuning using KerasTuner
def tune_lstm_model(generator):
    tuner = RandomSearch(
        build_lstm_model,
        objective='val_loss',
        max_trials=10,
        executions_per_trial=1,
        directory='lstm_tuner',
        project_name='lstm_sales')

    tuner.search(generator, epochs=50, validation_data=generator)

    # Get the best model and trial info
    best_model = tuner.get_best_models(num_models=1)[0]  # Return the best model
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]  # Get best trial

    # Print best model and trial information
    print("Best model summary:")
    best_model.summary()

    print(f"\nBest trial number: {best_trial.trial_id}")
    print("Best hyperparameters:")
    for param, value in best_trial.hyperparameters.values.items():
        print(f"{param}: {value}")

    return best_model

# Function to forecast using the LSTM model
def forecast_lstm(model, sales, scaler, n_input, n_features, forecast_horizon):
    forecast = []
    input_seq = scaler.transform(sales.values[-n_input:].reshape(-1, 1))  # Get last n_input data points

    for _ in range(forecast_horizon):
        input_seq = input_seq.reshape((1, n_input, n_features))  # Reshape for LSTM input
        prediction = model.predict(input_seq, verbose=0)  # Make prediction
        forecast.append(prediction[0, 0])  # Append prediction to forecast

        # Update the input sequence for the next prediction
        input_seq = np.append(input_seq[:, 1:, :], prediction.reshape((1, 1, n_features)), axis=1)  # Append predicted value

    return scaler.inverse_transform(np.array(forecast).reshape(-1, 1))  # Reverse scaling
# Function to handle outliers using the IQR method
def handle_outliers(df):
    Q1 = df['Volume'].quantile(0.25)
    Q3 = df['Volume'].quantile(0.75)
    IQR = Q3 - Q1

    # Identify outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace outliers with the median
    df['Volume'] = np.where(
        (df['Volume'] < lower_bound) | (df['Volume'] > upper_bound),
        df['Volume'].median(),
        df['Volume']
    )
    return df

# Function to parallel forecasting with SARIMA and LSTM
def parallel_hybrid_forecast(df, book_title, sarima_weight=0.5, lstm_weight=0.5):
    print(f"\nTraining and forecasting for {book_title}")

    # Handle outliers in the sales data
    df = handle_outliers(df)

    sales = df['Volume']
    sales_train = sales[:-forecast_horizon]  # Training data
    sales_test = sales[-forecast_horizon:]   # Test data

    # Fit SARIMA model
    sarima_model = sm.tsa.statespace.SARIMAX(sales_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 52))
    sarima_fit = sarima_model.fit(disp=False)

    # Get SARIMA predictions for the forecast horizon
    sarima_pred = sarima_fit.predict(start=len(sales_train), end=len(sales_train) + forecast_horizon - 1)

    # Prepare data for LSTM
    generator, scaler = prepare_data_lstm(sales_train, n_input, n_features)

    # Tune hyperparameters for LSTM
    best_lstm_model = tune_lstm_model(generator)

    # Train LSTM model
    best_lstm_model.fit(generator, epochs=50, validation_data=generator)

    # Forecast the next 32 weeks with LSTM
    lstm_forecast = forecast_lstm(best_lstm_model, sales_train, scaler, n_input, n_features, forecast_horizon)


    final_forecast = sarima_weight * sarima_pred.values + lstm_weight * lstm_forecast.flatten() # Combine predictions using weighted average

    # Calculate MAE and MAPE
    mae = mean_absolute_error(sales_test, final_forecast)
    mape = mean_absolute_percentage_error(sales_test, final_forecast) * 100  # Convert to percentage

    print(f"\nMean Absolute Error (MAE): {mae}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    # Plot the actual sales and final forecast
    plt.figure(figsize=(12, 6))

    # Plot original data
    plt.plot(sales.index, sales.values, label='Actual Sales', color='blue')

    # Plot final forecast
    future_index = pd.date_range(start=sales.index[-1], periods=forecast_horizon + 1, freq='W-SAT')[1:]
    plt.plot(future_index, final_forecast, label='Hybrid Forecast (SARIMA + LSTM)', color='green')

    plt.title(f"Sales Forecast for {book_title} using Parallel Hybrid SARIMA-LSTM Model")
    plt.xlabel("Date")
    plt.ylabel("Sales Volume")
    plt.legend()
    plt.show()

    # Show expected vs predicted values
    comparison_df = pd.DataFrame({'Actual Sales': sales_test.values, 'Forecasted Sales': final_forecast})
    print(comparison_df)

### The Alchemist

In [ ]:
# The Alchemist - Applying weights (sarima_weight=0.5, lstm_weight=0.5)
parallel_hybrid_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Alchemist, The'], 'The Alchemist', sarima_weight=0.5, lstm_weight=0.5)

In [ ]:
# The Alchemist - Applying weights (sarima_weight=0.4, lstm_weight=0.6)
parallel_hybrid_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Alchemist, The'], 'The Alchemist', sarima_weight=0.4, lstm_weight=0.6)

In [ ]:
# The Alchemist - Applying weights (sarima_weight=0.3, lstm_weight=0.7)
parallel_hybrid_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Alchemist, The'], 'The Alchemist', sarima_weight=0.3, lstm_weight=0.7)

### The Very Hungry Caterpillar

In [ ]:
# The Very Hungry Caterpillar - Applying weights (sarima_weight=0.5, lstm_weight=0.5)
parallel_hybrid_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Very Hungry Caterpillar, The'], 'The Very Hungry Caterpillar', sarima_weight=0.5, lstm_weight=0.5)

In [ ]:
# The Very Hungry Caterpillar - Applying weights (sarima_weight=0.7, lstm_weight=0.3)
parallel_hybrid_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Very Hungry Caterpillar, The'], 'The Very Hungry Caterpillar', sarima_weight=0.7, lstm_weight=0.3)

In [ ]:
# The Very Hungry Caterpillar - Applying weights (sarima_weight=0.8, lstm_weight=0.2)
parallel_hybrid_forecast(df_books_of_interest[df_books_of_interest['Title'] == 'Very Hungry Caterpillar, The'], 'The Very Hungry Caterpillar', sarima_weight=0.8, lstm_weight=0.2)

#Monthly Resampling

Next, resampling was performed to change the frequency into monthly and perform analysis. Since the sales data column required for the analysis is 'Volume', the aggregation operation (mean) was performed to 'Volume' and filled in the missing values with forward fill (ffill).

In [ ]:
# Resample to monthly frequency, mean of the 'Volume' column
monthly_df = (
    df_books_of_interest
    .groupby(['ISBN', 'Title'])  # Group by ISBN and Title
    .resample('ME')           # Resample to monthly frequency
    .agg({
        'Volume': 'mean'           # mean of the 'Volume' column
    })
    .ffill()
    .reset_index()                # Reset index to keep 'ISBN' and 'Title' as columns
)

# Display the final DataFrame
monthly_df

In [ ]:
# setting 'End Date' as index
monthly_df.set_index('End Date', inplace=True)
monthly_df

## XGBoost

Next trying out ML model, by creating the required pipeline for the XGBoost model. Performing cross - validation and parameter tuning (including window_length) using grid search. The forecast horizon is 8 months.

In [ ]:
# Create a function to create lagged features
def create_lagged_features(df, window_length):
    """Create lag and rolling mean features for the time series, handling NaN values."""
    df_lagged = pd.DataFrame()

    # Create lag features
    for i in range(1, window_length + 1):
        df_lagged[f'lag_{i}'] = df['Volume'].shift(i)

    # Create rolling mean features
    df_lagged['rolling_mean_3'] = df['Volume'].rolling(window=3).mean().shift(1)
    df_lagged['rolling_mean_6'] = df['Volume'].rolling(window=6).mean().shift(1)
    df_lagged['rolling_mean_12'] = df['Volume'].rolling(window=12).mean().shift(1)

    df_lagged['Volume'] = df['Volume']

    # Fill NaN values with the mean of the column
    df_lagged.fillna(df_lagged.mean(), inplace=True)

    return df_lagged

# Updated parameter grid to include regularization and tree parameters
param_grid = {
    'xgb__n_estimators': [100, 200],  # Number of trees
    'xgb__learning_rate': [0.01, 0.02, 0.03, 0.04, 0.1],  # Learning rate
    'xgb__max_depth': [3, 4, 5, 6],  # Maximum depth of a tree
    'xgb__subsample': [0.7, 0.8, 0.9, 1.0],  # Subsampling ratio
    'xgb__colsample_bytree': [0.7, 0.8, 0.9, 1.0],  # Subsampling ratio for columns
}

# Updated function to include outlier handling and more lags
def xgboost_forecast(df, book_title, window_lengths=[1, 3, 6, 7, 12]):
    print(f"\nTraining and forecasting for {book_title}")

    sales = df['Volume'].dropna()

    # Split data into training and testing
    train_data = sales[:-8]  # Keep the last 8 months as the test set
    test_data = sales[-8:]   # Test set is the last 8 months

    # Handle outliers by capping them
    Q1 = train_data.quantile(0.25)
    Q3 = train_data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    train_data = train_data.clip(lower=lower_bound, upper=upper_bound)

    best_score = float('inf')
    best_model = None
    best_window_length = None
    best_y_test = None
    best_y_pred = None

    for window_length in window_lengths:
        # Create lagged features for the training data
        df_train_lagged = create_lagged_features(train_data.to_frame(), window_length)
        X_train = df_train_lagged.drop('Volume', axis=1)
        y_train = df_train_lagged['Volume']

        # TimeSeriesSplit for cross-validation
        tscv = TimeSeriesSplit(n_splits=5)

        # Grid search with cross-validation
        grid_search = GridSearchCV(pipeline, param_grid, cv=tscv, scoring=scoring, refit='MAE', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Best model for the current window length
        best_model_current = grid_search.best_estimator_
        best_window_length_current = window_length

        # Create lagged features for the test data
        df_test_lagged = create_lagged_features(test_data.to_frame(), window_length)
        X_test = df_test_lagged.drop('Volume', axis=1)
        y_test = df_test_lagged['Volume']

        # Predict on the test data
        y_pred = best_model_current.predict(X_test)

        # Calculate MAE and MAPE
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)

        print(f"Window Length: {window_length} | MAE: {mae:.2f} | MAPE: {mape:.2f}%")

        # Store the best model and predictions if current one is better
        if mae < best_score:
            best_score = mae
            best_model = best_model_current
            best_window_length = best_window_length_current
            best_y_test = y_test
            best_y_pred = y_pred

    # Display expected and predicted values for the best window length
    print(f"\nExpected vs Predicted values for {book_title}:")
    expected_vs_predicted = pd.DataFrame({
        'Expected': best_y_test.values,
        'Predicted': best_y_pred
    })
    print(expected_vs_predicted)

    # Best MAE and MAPE
    best_mape = mean_absolute_percentage_error(best_y_test, best_y_pred)  # Calculate MAPE for best model
    print(f"\nBest Window Length: {best_window_length} | Best MAE: {best_score:.2f} | Best MAPE: {best_mape:.2f}%")

    # Final forecasting on the test set using the best model
    print(f"\nBest window length for {book_title}: {best_window_length}")
    df_final_lagged = create_lagged_features(sales.to_frame(), best_window_length)
    X_final = df_final_lagged.drop('Volume', axis=1)
    y_final = df_final_lagged['Volume']

    forecast = best_model.predict(X_final[-8:])  # Predict the next 8 months

    # Generate future date indexes for the forecast
    forecast_index = pd.date_range(start=sales.index[-1] + pd.DateOffset(months=1), periods=8, freq='M')

    # Create a DataFrame for the forecast with future dates
    forecast_df = pd.DataFrame({
        'Forecast Date': forecast_index,
        'Forecasted data': forecast
    })

    # Plot the actual sales, predictions, and forecast
    plt.figure(figsize=(12, 6))

    # Plot the original data (actual sales)
    plt.plot(sales.index, sales.values, label='Original Data', color='blue')

    # Plot the forecasted values
    plt.plot(forecast_index, forecast, label='Forecasted Values', color='green')

    plt.title(f"Actual Sales and Forecast for {book_title}")
    plt.xlabel('Time')
    plt.ylabel('Sales Volume')
    plt.legend()
    plt.show()

    return forecast_df, best_model


### The Alchemist

In [ ]:
# The Alchemist
xgboost_forecast(monthly_df[monthly_df['Title'] == 'Alchemist, The'], 'The Alchemist')

### The Very Hungry Caterpillar

In [ ]:
# The Very Hungry Caterpillar
xgboost_forecast(monthly_df[monthly_df['Title'] == 'Very Hungry Caterpillar, The'], 'The Very Hungry Caterpillar')

## SARIMA

Next is training the SARIMA model (using Auto ARIMA) on this data. The forecast horizon is 8 months.

In [ ]:
def sarima_forecast_monthly(df, book_title):
    print(f"\nTraining and forecasting for {book_title}")

    sales = df['Volume']
    forecast_horizon = 8
    train_data = sales[:-forecast_horizon]
    test_data = sales[-forecast_horizon:]

    # Fit the Auto ARIMA model
    model = auto_arima(train_data, seasonal=True, m=12,
                       trace=True, error_action='ignore',
                       suppress_warnings=True)

    print(model.summary())

    # Make forecast
    forecast, conf_int = model.predict(n_periods=forecast_horizon, return_conf_int=True)

    # Calculate MAE and MAPE
    mae = mean_absolute_error(test_data, forecast)
    mape = mean_absolute_percentage_error(test_data, forecast) * 100  # Convert to percentage

    print(f"\nMean Absolute Error (MAE): {mae}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    # Plot the actual sales and forecast
    plt.figure(figsize=(12, 6))
    plt.plot(sales.index, sales.values, label='Actual Sales', color='blue')
    forecast_index = pd.date_range(start=sales.index[-forecast_horizon], periods=forecast_horizon, freq='M')
    plt.plot(forecast_index, forecast, label='Forecasted Sales', color='green')
    plt.fill_between(forecast_index,
                     conf_int[:, 0], conf_int[:, 1], color='green', alpha=0.3)
    plt.title(f"Sales Forecast for {book_title}")
    plt.xlabel("Date")
    plt.ylabel("Sales Volume")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Show expected vs predicted values
    comparison_df = pd.DataFrame({'Actual Sales': test_data.values, 'Predicted Sales': forecast.values})
    print(comparison_df)

### The Alchemist

In [ ]:
# The Alchemist
sarima_forecast_monthly(monthly_df[monthly_df['Title'] == 'Alchemist, The'], 'The Alchemist')

### The Very Hungry Caterpillar

In [ ]:
# The Very Hungry Caterpillar
sarima_forecast_monthly(monthly_df[monthly_df['Title'] == 'Very Hungry Caterpillar, The'], 'The Very Hungry Caterpillar')



> The weekly and monthly frequency data both gave out better results when used with suitable models. When analysing sales data in weekly frequency for both "The Alchemist" and "The Very Hungry Caterpillar," the XGBoost model demonstrated superior performance. It effectively captured the underlying trends and seasonality of the sales patterns, providing accurate forecasts that aligned closely with the actual data.
In contrast, when the sales data was transformed into a monthly frequency, the SARIMA model outperformed other models. It excelled at capturing the long-term trends and seasonal fluctuations in the sales data, making it the best choice for forecasting in this scenario.


